In [7]:
import pandas as pd
import numpy as np

## Index 物件
* pandas的index物件會負責裝載軸標籤以及其他的metadata(軸的名稱...)
* 建立Series或DataFrame使用的標籤陣列或序列會被轉成Index物件

In [2]:
obj = pd.Series(range(3), index=['a', 'b', 'c'])
index = obj.index

In [3]:
obj

a    0
b    1
c    2
dtype: int64

In [4]:
index

Index(['a', 'b', 'c'], dtype='object')

In [5]:
index[:2]

Index(['a', 'b'], dtype='object')

* <font color=yellow>Index是不可變物件(immutable)</font>

In [8]:
labels = pd.Index(np.arange(3))
labels

Int64Index([0, 1, 2], dtype='int64')

In [9]:
obj2 = pd.Series([1.5, -2.5, 0], index=labels)
obj2

0    1.5
1   -2.5
2    0.0
dtype: float64

In [10]:
obj2.index is labels

True

In [11]:
pop = {'Nevada': {2021: 2.4, 2002: 2.9},
       'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

frame3 = pd.DataFrame(pop)
frame3

,Nevada,Ohio
2021,2.4,NaN
2002,2.9,3.6
2000,NaN,1.5
2001,NaN,1.7


In [13]:
frame3.index.name = 'year'; frame3.columns.name = 'state'

* Index的特性除了像陣列外，也可以像固定長度的set一樣


In [14]:
frame3

state,Nevada,Ohio
year,,
2021,2.4,NaN
2002,2.9,3.6
2000,NaN,1.5
2001,NaN,1.7


In [15]:
frame3.columns

Index(['Nevada', 'Ohio'], dtype='object', name='state')

In [16]:
'Ohio' in frame3.columns

True

In [17]:
2003 in frame3.index

False

* pandas的index和python的set不同的地方在於: pandas可以有重複的值

In [18]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])
dup_labels

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')

## 部分Index方法和屬性
|方法|屬性|
|:-:|:-:|
|append|連接另外的Index物件，並產生新的Index物件|
|difference|計算Index間的差集|
|intersection|計算Index間的交集|
|union|計算Index間的聯集|
|isin|查看傳入的資料是否存在Index物件中，以布林陣列回傳|
|delete|取得刪除索引i處的元素後新Index物件|
|drop|取得刪除指定值後新Index物件|
|insert|取得在索引i處插入新元素後新Index物件|
|is_monotonic|如果每個元素都大於等於前面的元素的話，回傳True|
|is_unique|如果沒有重複值的話，回傳True|
|unique|取得Index中部重複的資料陣列|

## 5.2重要功能

## 重作索引

* reindex方法 -> 用來建立新物件時，附帶新索引資料

In [19]:
obj = pd.Series([4.5, 7.2, -5.3, 3.5], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.5
dtype: float64

* 呼叫pandas的reindex方法，將會用新的index重排資料，如果新index無法匹配既有index -> 用缺失值填充

In [20]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.5
d    4.5
e    NaN
dtype: float64

* 時間序列 -> 有順序的資料，在reindex時可能會想做一些內插或填值的動作 -> 使用method參數
* method中的ffill參數 -> 會參考前面的值作填值

In [21]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [23]:
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

* 如果是DataFrame物件 -> 可以對欄、列或兩者同時進行reindex
* 指回傳一個序列的話 -> 預設是列

In [31]:
frame = pd.DataFrame(np.arange(9).reshape(3, 3),
                     index=['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [39]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


* 使用columns關鍵字可以對指定欄重作索引

In [41]:
state = ['Texas', 'Utah', 'California']
frame3 = frame.reindex(columns=state)
frame3

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


## reindex函式參數
|參數|描述|
|:-:|:-:|
|index|使用新的序列作為index，序列可以是另外一個index實例或其他Python資料結構中類似序列的東西，輸出索引將會完全按照序列設定，不做任何複製的動作|
|method|內插(填值)方法，'ffill'會複製前一個值，'bfill'會複製後一個值|
|fill_value|替代值，在重作索引時若發生缺失值的狀況，就用替代值替代|
|limit|使用前一個值或後一個值做插值時，最多填到哪裡(幾個元素個數)|
|tolerance|做index不精確比對情況使用，配合前一個值或後面一個值差值，最多填到哪裡(絕對數值差距)|
|level|使用MultiIndex時，指定匹配哪一層index|
|copy|如果設定True的話，即使新舊index完全相同，也會進行底層資料複製，如果為False，在index相等時，不複製資料|

## 指定軸刪除資料
* 若以擁有刪去某一或多個軸的index陣列的話 -> 刪去某一或多個軸就很容易
* 搭配清理或邏輯使用 -> 可以使用drop -> 會回傳一個刪去特定軸資料的新物件

In [50]:
obj = pd.Series(np.arange(5), index=['a', 'b', 'c', 'd', 'e'])
obj

a    0
b    1
c    2
d    3
e    4
dtype: int32

In [51]:
new_obj = obj.drop('c')
new_obj

a    0
b    1
d    3
e    4
dtype: int32

In [52]:
obj.drop(['d', 'c'])

a    0
b    1
e    4
dtype: int32

* DataFrame的兩個軸都可以用指定的index方法刪除資料

In [53]:
data = pd.DataFrame(np.arange(16).reshape(4, 4), 
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [54]:
data.drop(['Ohio', 'Colorado'])  # axis = 0 -> default

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


* 如果axis=1或axis='columns' -> 可以刪去指定欄的資料

In [55]:
data.drop(['two'], axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [56]:
data.drop(['two', 'four'], axis='columns')

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


* 可以使用in-place參數來修改原始資料，不建立新物件 -> <font color=yellow>會抹去刪除的資料</font>

In [57]:
obj.drop(['c'], inplace=True)

In [58]:
obj

a    0
b    1
d    3
e    4
dtype: int32

## 索引、選擇和過濾

* Series做索引(obj[...])的動作與Numpy陣列的索引很相似 -> 但是Series可用index值，不一定要是整數

In [59]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [60]:
obj['b']

1.0

In [61]:
obj[1]

1.0

In [63]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [64]:
obj[['b', 'a', 'd']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [65]:
obj[[1, 3]]

b    1.0
d    3.0
dtype: float64

In [66]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

* 使用<font color=yellow>標籤</font>做索引 -> <font color=yellow>會包含尾端</font>

In [67]:
obj['b':'c']

b    1.0
c    2.0
dtype: float64

* 給值的話，就會修改Series中對應的區域

In [68]:
obj['b':'c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

* 對DataFrame做索引的話，使用單一或序列都可以，可取回一欄或多欄的值

In [69]:
data = pd.DataFrame(np.arange(16).reshape(4, 4), 
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [70]:
data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int32

In [71]:
data[['three', 'one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


In [72]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [73]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [74]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [75]:
data[data < 5] = 0

In [76]:
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


## 用loc和iloc做選擇
* 使用類似numpy的方法，配合軸標籤(loc)或整數(iloc)來做選擇
* loc做切片 -> 會包含尾端
* iloc做切片 -> 不會包含尾端

* 選擇一個列及多個欄

In [79]:
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [77]:
data.loc['Colorado', ['two', 'three']]

two      5
three    6
Name: Colorado, dtype: int32

* 改用iloc做類似的選擇動作

In [78]:
data.iloc[2, [3, 0, 1]]

four    11
one      8
two      9
Name: Utah, dtype: int32

In [80]:
data.iloc[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int32

In [81]:
data.iloc[[1, 2], [3, 0, 1]]

,four,one,two
Colorado,7,0,5
Utah,11,8,9


* 除了用一或多個標籤做選擇外，兩種索引方法都支援切片

In [82]:
data.loc[:'Utah', 'two']

Ohio        0
Colorado    5
Utah        9
Name: two, dtype: int32

In [85]:
data.iloc[:, :3][data['three'] > 5]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


## DataFrame的索引方法
|方法|附註|
|:-:|:-:|
|df[value]|從DataFrame中選擇一個或連續的欄，在以下情況使用很方便: 布林陣列(過濾列)，切片(切片列)或布林DataFrame(在條件成立時設定值)|
|df.loc[value]|從DataFrame中用標籤選擇一到多個列|
|df.loc[ : ,  value]|用標籤選擇一到多個列|
|df.loc[value1, value2]|用標籤選擇欄或列|
|df.iloc[where]|從DataFrame中用整數選擇一到多個列|
|df.iloc[ : ,  where]|用整數位置選擇一到多個欄|
|df.iloc[label_i, label_j]|用整數選擇欄或列|
|df.at[i, j]|用欄和列標籤選擇單一值|
|df.iat[i, j]|用欄和列位置(整數)選擇單一值|
|reindex方法|用標籤選擇欄或列|
|get_value, set_value方法|用欄和列標籤選擇單一值|

## 整數索引
* 與tuple和list的索引方法有些許差異

In [86]:
ser = pd.Series(np.arange(3.))
ser

0    0.0
1    1.0
2    2.0
dtype: float64

In [88]:
ser[-1]  # ->會報錯，因為使用此方法可能會與整數index產生混淆 -> 可將index換成非整數 

KeyError: -1

In [90]:
ser2 = pd.Series(np.arange(3.), index=['a', 'b', 'c'])
ser2

a    0.0
b    1.0
c    2.0
dtype: float64

In [91]:
ser2[-1]

2.0

* 切片

In [92]:
ser[:1]

0    0.0
dtype: float64

In [94]:
ser.loc[:1]  # loc切片會包含尾端

0    0.0
1    1.0
dtype: float64

In [95]:
ser.iloc[:1]  # iloc切片不會包含尾端

0    0.0
dtype: float64

## 算術運算與資料對齊
* 若是兩個pandas物件做算術運算時，index皆無重複 -> 產生後的index為兩者的index聯集 -> like SQL outer join語法

In [96]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [97]:
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1], index=['a', 'c', 'e', 'f', 'g'])
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [98]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

* 內部的資料對齊功能使得沒有配對的標籤處產生了缺失值，這些缺失值在之後做其他算術運算時將會傳播

* 在使用DataFrame時，全部的欄或列都會進行對齊動作

In [99]:
df1 = pd.DataFrame(np.arange(9.).reshape(3, 3),
                   columns=list('bcd'),
                   index=['Ohio', 'Texas', 'Colorado'])
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [100]:
df2 = pd.DataFrame(np.arange(12.).reshape(4, 3),
                   columns=list('bde'),
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


* 將兩個物件相加 -> 回傳一個DataFrame，這個DataFrame的index和欄位是原本兩個DataFrame做聯集的結果

In [101]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


* 因為'c'和'e'欄沒有同時存在兩個DataFrame中 -> 整欄都是缺失值
* 同理，標籤若是無法兩個DataFrame都找到 -> 整列都是缺失值

* 如果將兩個沒有共同欄標籤或共同列標籤的DataFrame做運算的話 -> 產出全為null的DataFrame

In [102]:
df1 = pd.DataFrame({"A": [1, 2]})
df1

,A
0,1
1,2


In [104]:
df2 = pd.DataFrame({'B': [3, 4]})
df2

,B
0,3
1,4


In [105]:
df1 - df2

,A,B
0,NaN,NaN
1,NaN,NaN
